In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import sum as pyspark_sum

In [2]:
spark = SparkSession.builder.appName('mySpark').master('local[*]').getOrCreate()
spark

In [3]:
df = spark.read.csv('US_Accidents_March23.csv', header = True, inferSchema = True)
df.show(10)

+----+-------+--------+-------------------+-------------------+------------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  ID| Source|Severity|         Start_Time|           End_Time|         Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|A

In [4]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

In [5]:
null = [pyspark_sum(f.when(f.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
df_null = df.agg(*null)
df_null.show()

+---+------+--------+----------+--------+---------+---------+-------+-------+------------+-----------+------+----+------+-----+-------+-------+--------+------------+-----------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+----+--------+--------+--------+-------+-------+----------+-------+----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
| ID|Source|Severity|Start_Time|End_Time|Start_Lat|Start_Lng|End_Lat|End_Lng|Distance(mi)|Description|Street|City|County|State|Zipcode|Country|Timezone|Airport_Code|Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity|Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station|Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_Sunset|Civil_Twilight|Nautical_Twil

In [6]:
df = df.select('ID', 'Severity', 'Start_Time', 'End_Time', 'Description', 'Street', 'City', 'County', 'State', 'Timezone',\
            'Weather_Timestamp', 'Temperature(F)', 'Humidity(%)', 'Visibility(mi)', 'Weather_Condition')
df.show(10)

+----+--------+-------------------+-------------------+--------------------+--------------------+------------+----------+-----+----------+-------------------+--------------+-----------+--------------+-----------------+
|  ID|Severity|         Start_Time|           End_Time|         Description|              Street|        City|    County|State|  Timezone|  Weather_Timestamp|Temperature(F)|Humidity(%)|Visibility(mi)|Weather_Condition|
+----+--------+-------------------+-------------------+--------------------+--------------------+------------+----------+-----+----------+-------------------+--------------+-----------+--------------+-----------------+
| A-1|       3|2016-02-08 05:46:00|2016-02-08 11:00:00|Right lane blocke...|              I-70 E|      Dayton|Montgomery|   OH|US/Eastern|2016-02-08 05:58:00|          36.9|       91.0|          10.0|       Light Rain|
| A-2|       2|2016-02-08 06:07:59|2016-02-08 06:37:59|Accident on Brice...|            Brice Rd|Reynoldsburg|  Franklin|   

In [7]:
df= df.withColumn('Duration_Hr', (f.col('End_Time').cast('long') - f.col('Start_Time').cast('long')) / 3600)
df.show(10)    

+----+--------+-------------------+-------------------+--------------------+--------------------+------------+----------+-----+----------+-------------------+--------------+-----------+--------------+-----------------+-----------------+
|  ID|Severity|         Start_Time|           End_Time|         Description|              Street|        City|    County|State|  Timezone|  Weather_Timestamp|Temperature(F)|Humidity(%)|Visibility(mi)|Weather_Condition|      Duration_Hr|
+----+--------+-------------------+-------------------+--------------------+--------------------+------------+----------+-----+----------+-------------------+--------------+-----------+--------------+-----------------+-----------------+
| A-1|       3|2016-02-08 05:46:00|2016-02-08 11:00:00|Right lane blocke...|              I-70 E|      Dayton|Montgomery|   OH|US/Eastern|2016-02-08 05:58:00|          36.9|       91.0|          10.0|       Light Rain|5.233333333333333|
| A-2|       2|2016-02-08 06:07:59|2016-02-08 06:37:

In [8]:
df = df.withColumn('Occurrence_day', f.to_date(f.col('Start_Time')))
df.show(5)


+---+--------+-------------------+-------------------+--------------------+--------------------+------------+----------+-----+----------+-------------------+--------------+-----------+--------------+-----------------+-----------------+--------------+
| ID|Severity|         Start_Time|           End_Time|         Description|              Street|        City|    County|State|  Timezone|  Weather_Timestamp|Temperature(F)|Humidity(%)|Visibility(mi)|Weather_Condition|      Duration_Hr|Occurrence_day|
+---+--------+-------------------+-------------------+--------------------+--------------------+------------+----------+-----+----------+-------------------+--------------+-----------+--------------+-----------------+-----------------+--------------+
|A-1|       3|2016-02-08 05:46:00|2016-02-08 11:00:00|Right lane blocke...|              I-70 E|      Dayton|Montgomery|   OH|US/Eastern|2016-02-08 05:58:00|          36.9|       91.0|          10.0|       Light Rain|5.233333333333333|    2016-02-

In [9]:
df = df.withColumn('Weather_Timestamp', f.to_date(f.col('Weather_Timestamp')))
media_clima = df.groupBy('Weather_Timestamp').agg(
    {   
     'Temperature(F)': 'avg',
     'Humidity(%)': 'avg',
     'Visibility(mi)': 'avg'
    }
    ).orderBy('Weather_Timestamp')
media_clima.show()

+-----------------+-------------------+-------------------+------------------+
|Weather_Timestamp|avg(Visibility(mi))|avg(Temperature(F))|  avg(Humidity(%))|
+-----------------+-------------------+-------------------+------------------+
|             NULL|               NULL|               NULL|              NULL|
|       2016-01-14|               10.0|               31.0|              69.0|
|       2016-02-08|               7.13| 35.388333333333335| 91.81666666666666|
|       2016-02-09|  2.410169491525424|  23.53220338983051| 87.11864406779661|
|       2016-02-10|  3.875510204081633|  17.76530612244898| 77.57142857142857|
|       2016-02-11|  9.263440860215054| 15.888172043010755| 65.31182795698925|
|       2016-02-12|  8.416666666666666|  19.20555555555556| 66.33333333333333|
|       2016-02-13|               10.0| 10.142857142857142|58.357142857142854|
|       2016-02-14|  5.827272727272727|  13.19090909090909| 80.18181818181819|
|       2016-02-15| 3.8548387096774195| 26.927419354

In [12]:
acidentes = df.groupBy('Occurrence_day', 'State', 'County', 'City', 'Street', 'Severity').count().orderBy('Occurrence_day')
acidentes.show()

+--------------+-----+----------+------------+---------------+--------+------------------+-----+
|Occurrence_day|State|    County|        City|         Street|Severity|       Duration_Hr|count|
+--------------+-----+----------+------------+---------------+--------+------------------+-----+
|    2016-01-14|   PA|    Lehigh|   Whitehall|        US-22 E|       4| 9161.112777777778|    1|
|    2016-01-14|   PA|    Lehigh|   Whitehall|        US-22 E|       4|         9603.0575|    1|
|    2016-01-14|   PA|    Lehigh|   Whitehall|        US-22 E|       4| 9161.619722222222|    1|
|    2016-01-14|   PA|    Lehigh|   Whitehall|        US-22 E|       4| 9629.324166666667|    1|
|    2016-01-14|   PA|    Lehigh|   Whitehall|        US-22 E|       4| 9161.258055555556|    1|
|    2016-01-14|   PA|    Lehigh|   Whitehall|        US-22 E|       4|10275.508333333333|    1|
|    2016-01-14|   PA|    Lehigh|   Whitehall|        US-22 E|       4|10553.135277777777|    1|
|    2016-02-08|   OH|  Clermo

In [11]:
acidentes = acidentes.join(media_clima, on = media_clima['Weather_Timestamp'] == acidentes['Occurrence_day'], how = 'outer').orderBy('Occurrence_day')
acidentes.show()

+--------------+-----+----------+------------+----------------+--------+-----+-----------------+-------------------+-------------------+-----------------+
|Occurrence_day|State|    County|        City|          Street|Severity|count|Weather_Timestamp|avg(Visibility(mi))|avg(Temperature(F))| avg(Humidity(%))|
+--------------+-----+----------+------------+----------------+--------+-----+-----------------+-------------------+-------------------+-----------------+
|          NULL| NULL|      NULL|        NULL|            NULL|    NULL| NULL|             NULL|               NULL|               NULL|             NULL|
|    2016-01-14|   PA|    Lehigh|   Whitehall|         US-22 E|       4|    7|       2016-01-14|               10.0|               31.0|             69.0|
|    2016-02-08|   OH|  Clermont|     Batavia|  State Route 32|       2|    1|       2016-02-08|               7.13| 35.388333333333335|91.81666666666666|
|    2016-02-08|   OH|  Franklin|    Columbus|    Brookhill Dr|       